# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
# Initialization

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
# MODEL = "llama3.2"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


In [ ]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."
#system_message += "Always be accurate."

In [ ]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True)

## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [ ]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [ ]:
get_ticket_price("Berlin")

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

def extract_webpage_text(url):
    """
    Extracts clean text content from a webpage, excluding links, images, scripts, and other HTML elements.
    
    Args:
        url (str): The URL of the webpage to scrape
        
    Returns:
        str: Clean text content from the webpage
        
    Raises:
        requests.RequestException: If there's an error fetching the webpage
        ValueError: If the URL is invalid
    """
    try:
        # Send request with a common user agent to avoid blocks
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Remove unwanted elements
        for element in soup(['script', 'style', 'head', 'title', 'meta', '[document]']):
            element.decompose()
            
        # Remove all links but keep their text content
        for link in soup.find_all('a'):
            link.unwrap()
            
        # Get text and clean it up
        text = soup.get_text()
        
        # Clean up whitespace
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = ' '.join(chunk for chunk in chunks if chunk)
        
        # Remove multiple spaces and special characters
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'[^\x00-\x7F]+', '', text)  # Remove non-ASCII characters
        
        return text.strip()
        
    except requests.RequestException as e:
        raise requests.RequestException(f"Error fetching webpage: {str(e)}")
    except ValueError as e:
        raise ValueError(f"Invalid URL provided: {str(e)}")

In [ ]:
#extract_webpage_text("https://www.lonelyplanet.com/search?q=berlin")

# This use case less useful, because the data is fairly static.
extract_webpage_text("https://wikitravel.org/en/berlin")


In [ ]:
# This use case is useful, because its a REAl TIME API call.

import requests
import random


def get_price_from_api(city):
    print(f">>>>> Tool [get_price_from_api] called for \"{city}\"")

    try:
        product_id = str(random.randint(1, 189))
        url = 'https://dummyjson.com/products/' + product_id
        response = requests.get(url)
        print(f"Called {url} ---> {response}")
        response.raise_for_status()  # Raises an exception for bad status codes
        data = response.json()
#        print(json.dumps(data, indent=4))
        return data['price']
        
    except requests.exceptions.RequestException as e:
        print(f"Error making request: {e}")
    except ValueError as e:
        print(f"Error parsing JSON: {e}")

In [ ]:
get_price_from_api("berlin")


In [ ]:
# There's a particular dictionary structure that's required to describe our function:

price_api_function = {
    "name": "get_price_from_api",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
# And this is included in a list of tools:

#tools = [{"type": "function", "function": price_function}]
tools = [{"type": "function", "function": price_api_function}]

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    print(f"\n\n\nCURRENT MESSAGE STACK >>: \n\n")
    for msg in messages:
        print(f"\n{msg}\n")
    print(f"\n ---- END (current stack) ----\n")

    
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    finish_reason = response.choices[0].finish_reason
    print(f"The response finished because: {finish_reason}")

#    if response.choices[0].finish_reason=="tool_calls":
    if finish_reason=="tool_calls":
        message_from_model = response.choices[0].message
        tool_call_id = message_from_model.tool_calls[0].id
        print("*** The Model wants us to make an external API call  ==")
        print(f"*** TOOL CALL with ID = {tool_call_id}              ==")
        response, city = handle_tool_call(message_from_model)
        print(f"The response, city: {response,city}")

        # IMPORTANT - here we return the request & response to the model so it can access the result.
        # Each has the tool_call_id embedded so the Model knows to connect them.
        messages.append(message_from_model)
        messages.append(response)

        print(f"\n\n\nTOOL CALLS MESSAGE >>: \n\n")
        for msg in messages:
            print(f"\n{msg}\n")
        print("---- END (tool_calls stack) ----")
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
# We have to write that function handle_tool_call:

def handle_tool_call(message_from_model):
    tool_call = message_from_model.tool_calls[0]
    print(f"tool_call is ======> {tool_call}")

    # Here we could test for different tool_calls functons:
    if tool_call.function.name == 'get_price_from_api':        
        print(f"Executing tool_call for [get_price_from_api]...")
    
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
    
        print(f"Received args from Model: {json.dumps(arguments, indent=2)}")
        print("*** ===================================================")
    #    price = get_ticket_price(city)

        # Call to ext API!  (This is the juicy part!)
        price = get_price_from_api(city)

        # Pack response to Model (with result and id):
        response = {
            "role": "tool", # New role! system, user, assistant, tool
            "content": json.dumps({"destination_city": city,"price": price}),
            "tool_call_id": tool_call.id # id required for messages with role 'tool'
        }
        return response, city

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True)